<a href="https://colab.research.google.com/github/Haruw09/MATLAB_2023/blob/main/Colab_MD_water_HW_dipeptide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installation of GROMACS**

https://manual.gromacs.org/2024.0/index.html

https://doi.org/10.1016/j.softx.2015.06.001

In [1]:
#Downloading GROMACS 2024

!wget https://ftp.gromacs.org/gromacs/gromacs-2024.tar.gz

--2024-03-26 15:51:26--  https://ftp.gromacs.org/gromacs/gromacs-2024.tar.gz
Resolving ftp.gromacs.org (ftp.gromacs.org)... 130.237.11.165, 2001:6b0:1:1191:216:3eff:fec7:6e30
Connecting to ftp.gromacs.org (ftp.gromacs.org)|130.237.11.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42455653 (40M) [application/x-gzip]
Saving to: ‘gromacs-2024.tar.gz’

gromacs-2024.tar.gz 100%[===================>]  40.49M  16.1MB/s    in 2.5s    

2024-03-26 15:51:29 (16.1 MB/s) - ‘gromacs-2024.tar.gz’ saved [42455653/42455653]



In [2]:
%%bash

#Extracting and Installing GROMACS 2024 onto a user-defined folder
#First, we extract the downloaded folder containing GROMACS-2024

tar xfz gromacs-2024.tar.gz # tar - archive, gz - gzip compression

echo "GROMACS extraction completed"

#Then, we create a "build" folder inside the extracted folder

cd gromacs-2024

mkdir build

#Next, we setup our GROMACS 2024 installation options, including GPU and a user-defined folder

cd build

cmake .. -DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=CUDA -DCMAKE_INSTALL_PREFIX=/content/gromacs-24
echo "GROMACS set up completed"

GROMACS extraction completed
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python3: /usr/local/bin/python (found suitable version "3.10.12", minimum required is "3.7") found components: Interpreter Development Development.Module Development.Embed 
-- Selected GPU FFT library - cuFFT
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Performing Test CFLAGS_WARN_NO_MISSING_FIELD_INITIALIZERS
-- Performing Test CFLAGS_WARN_NO_MIS

In [ ]:
%%bash

cd /content/gromacs-2024/build

make
echo "GROMACS building completed"

#make check
#echo "GROMACS testing completed"

make install
echo "GROMACS installation completed. Please check if any errors occurred during installation"

In [ ]:
%%bash

##Checking that GROMACS was successfully installed

source /content/gromacs-24/bin/GMXRC

gmx -h

In [ ]:
#First, we will mount your Google Drive to be able to transfer files
#You will be requested to authorize this Google Drive mounting

from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
#Copying your compiled GROMACS to your Google Drive
#We will create and/or use the MDSim folder to create a folder for compiled programs

import os
import shutil
from pathlib import Path

MDSim = Path("/content/gdrive/My Drive/MDSim")

if os.path.exists(MDSim):
  print("MDSim already exists")
if not os.path.exists(MDSim):
  os.mkdir(MDSim)
  print("MDSim did not exists and was succesfully created")

#Then, we will copy the compiled GROMACS to this folder

#shutil.copytree(str('/content/gromacs-2024'), str(MDSim/'gromacs-2024'))
shutil.copytree(str('/content/gromacs-24'), str(MDSim/'gromacs-24'))

#!cp -r /content/gromacs-24 "$MDSimulations"/gromacs-24

print("GROMACS successfully backed up!")

In [ ]:
#%%bash

#wget https://zenodo.org/records/10822792/files/gromacs-24.zip

#unzip gromacs-24.zip

**Setting up the MD simulation system: TIP3P water box**

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

#we will create a water box
gmx solvate -cs spc216.gro -o wat.gro -box 2.5 2.5 2.5

#creating structure and topology for TIP3P water
gmx pdb2gmx -f wat.gro -o wat.gro -p syst.top -ignh -ff charmm27 -water tip3p

# creating tpr for genion
wget https://zenodo.org/records/10822792/files/ions.mdp

gmx grompp -f ions.mdp -c wat.gro -p syst.top -o ions.tpr

#Using genion and the tpr to add counterions to our solvated system
echo "SOL" > options
gmx genion -s ions.tpr -o wat_ions.gro -p syst.top -pname NA -nname CL -conc 0.15 -neutral < options


The **force field** - a collection of equations and associated
constants designed to reproduce molecular geometry and selected
properties of tested structures.

**mdp** - molecular dynamics parameters
https://manual.gromacs.org/documentation/current/user-guide/mdp-options.html

**tpr** - contains the starting structure of your simulation, the molecular topology and all the simulation parameters

**gro** - plain text file storing spatial coordinates and velocities

**top** - defines which atoms are connected to one another through chemical bond

**edr** - energies, temperature, pressure, box size, density and virials



**Energy minimization**

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

wget https://zenodo.org/records/10822792/files/min.mdp

gmx grompp -f min.mdp -c wat_ions.gro -p syst.top -o min.tpr

gmx mdrun -deffnm min -nb gpu

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

#Using energy to extract the potential energy of the system
echo "Potential" > options
gmx energy -f min.edr -o min_potential.xvg < options

In [ ]:
import numpy as np
import matplotlib.pyplot as pp
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

#Plotting the potential energy of the system
#Reading the text file containing this information

data = np.loadtxt(str('min_potential.xvg'), skiprows=25)

pp.title('Potential Energy during Minimization')
pp.xlabel('Energy Minimization Step')
pp.ylabel(r'E$_P$ [kJ•mol$^{-1}]$')
pp.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
pp.show()

The minimization is converged when the maximum force is smaller than *emtol* (in .mdp) value

**Equilibration under an NVT ensemble**

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

wget https://zenodo.org/records/10822792/files/eq_nvt_1fs.mdp

gmx grompp -f eq_nvt_1fs.mdp -c min.gro -p syst.top -o eq_nvt_1fs.tpr -maxwarn 1

gmx mdrun -deffnm eq_nvt_1fs -nb gpu

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

#Using energy to extract the temperature of the system during the NVT equil MD
echo "Temperature" > options
gmx energy -f eq_nvt_1fs.edr -o eq_nvt_1fs_temp.xvg < options

In [ ]:
#Plotting the temperature of the system
import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

data = np.loadtxt(str('eq_nvt_1fs_temp.xvg'), skiprows=25)

pp.title('Temperature during 50 ps Equilibration (NVT)')
pp.xlabel('Time (ps)')
pp.ylabel('Temperature [K]')
pp.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
pp.show()

**Equilibration under an NPT ensemble**

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

wget https://zenodo.org/records/10822792/files/eq_npt_1fs.mdp

gmx grompp -f eq_npt_1fs.mdp -c eq_nvt_1fs.gro -p syst.top -o eq_npt_1fs.tpr -maxwarn 2

gmx mdrun -deffnm eq_npt_1fs -nb gpu # 100 ps

###############

wget https://zenodo.org/records/10822792/files/eq_npt_2fs.mdp

gmx grompp -f eq_npt_2fs.mdp -c eq_npt_1fs.gro -p syst.top -o eq_npt_2fs.tpr -maxwarn 2

gmx mdrun -deffnm eq_npt_2fs -nb gpu # 10 ns

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

#Using energy to extract the pressure and density of the system during the NPT equil MD
echo "Pressure" > options
echo "Density" >> options
gmx energy -f eq_npt_2fs.edr -o npt_press_dens.xvg < options

Pressure

In [ ]:
import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

data = np.loadtxt(str('npt_press_dens.xvg'), skiprows=25)

print('Pressure = ',"%.1f" % np.mean(data[:,1]), '±', "%.1f" % np.std(data[:,1]), 'bar')

pp.title('Pressure during 10 ns Equilibration (NPT)')
pp.xlabel('Time (ps)')
pp.ylabel('Pressure (bar)')
pp.plot(data[:,0], data[:,1], linestyle='solid', linewidth='1', color='red')
pp.show()

Density

In [ ]:
import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

data = np.loadtxt(str('npt_press_dens.xvg'), skiprows=25)
print('Density = ',"%.1f" % np.mean(data[:,2]), '±', "%.1f" % np.std(data[:,2]), '1/bar')

pp.title('Density during 10 ns Equilibration (NPT)')
pp.xlabel('Time (ps)')
pp.ylabel('Density (kg•m$^{-3}$)')
pp.plot(data[:,0], data[:,2], linestyle='solid', linewidth='1', color='red')
pp.show()

**MD production**

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

wget https://zenodo.org/records/10822792/files/prod.mdp

gmx grompp -f prod.mdp -c eq_npt_2fs.gro -p syst.top -o prod.tpr

gmx mdrun -deffnm prod -nb gpu # 10 ns

**Trajectory analysis**

Comparison of temperature and kinetic energy distributions for different thermostats

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

#Using energy to extract the pressure and density of the system during the NPT prod MD
echo "Temperature" > options
echo "Kinetic-En." >> options
gmx energy -f eq_npt_2fs.edr -o npt_temp_kin.xvg < options # Berendsen
gmx energy -f prod.edr -o prod_temp_kin.xvg < options # Nose-Hoover

Temperature and Kinetic Energy distribution

In [ ]:
import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

ber = np.loadtxt(str('npt_temp_kin.xvg'), skiprows=25) # Berendsen
n_h = np.loadtxt(str('prod_temp_kin.xvg'), skiprows=25) # Nose-Hoover

print('Ber = ',"%.1f" % np.mean(ber[:,2]), '±', "%.1f" % np.std(ber[:,2]), 'K')
print('N-H = ',"%.1f" % np.mean(n_h[:,2]), '±', "%.1f" % np.std(n_h[:,2]), 'K')

print('Ber = ',"%.1f" % np.mean(ber[:,1]), '±', "%.1f" % np.std(ber[:,1]), 'kJ/mol')
print('N-H = ',"%.1f" % np.mean(n_h[:,1]), '±', "%.1f" % np.std(n_h[:,1]), 'kJ/mol')

pp.title('Comparison of Kinetic Energy')
pp.xlabel('Time (ps)')
pp.ylabel('Kinetic Energy (kJ/mol)')
pp.plot(ber[:,0], ber[:,1], linestyle='solid', linewidth='1', color='red', label='Berendsen')
pp.plot(n_h[:,0], n_h[:,1], linestyle='solid', linewidth='1', alpha=0.5, color='blue', label='Nose-Hoover')
pp.legend()
pp.show()

Pressure

https://manual.gromacs.org/current/user-guide/terminology.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

#Using energy to extract the pressure and density of the system during the NPT prod MD
echo "Pressure" > options
echo "Density" >> options
gmx energy -f prod.edr -o prod_press_dens.xvg < options

In [ ]:
import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

ber = np.loadtxt(str('npt_press_dens.xvg'), skiprows=25) # Berendsen
p_r = np.loadtxt(str('prod_press_dens.xvg'), skiprows=25) # Parrinello-Rahman

print('Ber = ',"%.1f" % np.mean(ber[:,1]), '±', "%.1f" % np.std(ber[:,1]), 'bar')
print('P-R = ',"%.1f" % np.mean(p_r[:,1]), '±', "%.1f" % np.std(p_r[:,1]), 'bar')

pp.title('Pressure')
pp.xlabel('Time (ps)')
pp.ylabel('Pressure [bar]')
pp.plot(ber[:,0], ber[:,1], linestyle='solid', linewidth='1', color='red', label='Berendsen')
pp.plot(p_r[:,0], p_r[:,1], linestyle='solid', linewidth='1', alpha=0.5, color='blue', label='Parrinello-Rahman')
pp.legend()
pp.show()

Density

In [ ]:
import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

ber = np.loadtxt(str('npt_press_dens.xvg'), skiprows=25) # Berendsen
p_r = np.loadtxt(str('prod_press_dens.xvg'), skiprows=25) # Parrinello-Rahman

print('Ber = ',"%.1f" % np.mean(ber[:,2]), '±', "%.1f" % np.std(ber[:,2]), 'kg/m^3')
print('P-R = ',"%.1f" % np.mean(p_r[:,2]), '±', "%.1f" % np.std(p_r[:,2]), 'kg/m^3')

Trajectory postprocessing

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

echo "System" >> options
gmx trjconv -f prod.xtc -o prod.xtc -pbc mol -s prod.tpr < options

Creating an index file

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

echo "splitres 4" > options
echo "ri 1" >> options
echo "2 & ! ri 1" >> options
echo "q" >> options
gmx make_ndx -f prod.gro -o ind.ndx < options

Radial distribution function

https://manual.gromacs.org/2024.0/reference-manual/analysis/radial-distribution-function.html



In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

# water around a water molecule
echo "8" > options # 1 water molecule
echo "9" >> options # water without "8"
gmx rdf -f prod.xtc -s prod.tpr -n ind.ndx -o rdf_wat1.xvg -pbc < options

# water around NA+
echo "6" > options # ion NA+
echo "2" >> options # water
gmx rdf -f prod.xtc -s prod.tpr -n ind.ndx -o rdf_na.xvg -pbc < options

# water around CL-
echo "7" > options # ion CL-
echo "2" >> options # water
gmx rdf -f prod.xtc -s prod.tpr -n ind.ndx -o rdf_cl.xvg -pbc < options

# CL- around NA+
#echo "6" > options # ion NA+
#echo "7" >> options # ion CL-
#gmx rdf -f prod.xtc -s prod.tpr -n ind.ndx -o rdf_na_cl.xvg -pbc < options

In [ ]:
import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

na = np.loadtxt(str('rdf_na.xvg'), skiprows=25)*10
cl = np.loadtxt(str('rdf_cl.xvg'), skiprows=25)*10
w = np.loadtxt(str('rdf_wat1.xvg'), skiprows=25)*10

#na_cl = np.loadtxt('rdf_na_cl.xvg', skiprows=25)*10

pp.title('RDF')
pp.xlabel('Distance, r (Å)')
pp.ylabel('g(r)')
pp.plot(na[:,0], na[:,1], linestyle='solid', linewidth='1', color='blue', label="around NA+")
pp.plot(cl[:,0], cl[:,1], linestyle='solid', linewidth='1', color='red', label="around CL-")
pp.plot(w[:,0], w[:,1], linestyle='solid', linewidth='1', color='black', label="around 1 H2O")

#pp.plot(na_cl[:,0], na_cl[:,1], linestyle='solid', linewidth='1', color='orange', alpha=0.5,label="CL- around NA+")
#pp.ylim(0,40)
#pp.plot(p_r[:,0], p_r[:,1], linestyle='solid', linewidth='1', alpha=0.5, color='blue')

pp.legend(loc='upper right')
pp.show()

Mean squared displacement and Diffusion

https://manual.gromacs.org/2024.0/reference-manual/analysis/mean-square-displacement.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

echo "2" > options # water
gmx msd -f prod.xtc -s prod.tpr -n ind.ndx -pbc -mol diff_w.xvg -o msd_w.xvg < options

echo "6" > options # ion NA+
gmx msd -f prod.xtc -s prod.tpr -n ind.ndx -pbc -mol diff_na.xvg -o msd_na.xvg < options

echo "7" > options # ion CL-
gmx msd -f prod.xtc -s prod.tpr -n ind.ndx -pbc -mol diff_cl.xvg -o msd_cl.xvg < options

In [ ]:
import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

w = np.loadtxt(str('msd_w.xvg'), skiprows=25)
na = np.loadtxt(str('msd_na.xvg'), skiprows=25)
cl = np.loadtxt(str('msd_cl.xvg'), skiprows=25)

print("D (TIP3P water) = ","5.6743 ± 0.0507 (e-5 cm^2/s)") # from MSD curve

pp.title('MSD')
pp.xlabel('Time lag (τ)')
pp.ylabel('MSD (nm$^2$/N)')
pp.plot(w[:,0], w[:,1], linestyle='solid', linewidth='1', color='black', label="water")
pp.plot(na[:,0], na[:,1], linestyle='solid', linewidth='1', color='blue', label="NA+")
pp.plot(cl[:,0], cl[:,1], linestyle='solid', linewidth='1', color='red', label="CL-")
pp.legend()
pp.show()

In [ ]:
import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/tip3p")

w = np.loadtxt(str('diff_w.xvg'), skiprows=25) # for each molecule

pp.title('MSD')
pp.xlabel('Molecule (#)')
pp.ylabel('Diffusion (e-5 cm^2/s)')
pp.plot(w[:,0], w[:,1], linestyle='solid', linewidth='1', color='red')
pp.show()

**OPC water box**

Generated by *CHARMM-GUI*

https://doi.org/10.1002/jcc.20945

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import shutil
from pathlib import Path
import os

opc = Path("/content/gdrive/My Drive/opc")

shutil.copytree(str(opc), str('/content/opc'))

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/opc

# Minimization
gmx grompp -f min.mdp -c wat_ions.gro -p syst.top -o min.tpr
gmx mdrun -v -deffnm min -nb gpu

#Thermalization and Equilibration
# NVT
gmx grompp -f eq_nvt_1fs.mdp -c min.gro -p syst.top -o eq_nvt_1fs.tpr -maxwarn 1
gmx mdrun -deffnm eq_nvt_1fs -nb gpu
# NPT
gmx grompp -f eq_npt_1fs.mdp -c eq_nvt_1fs.gro -p syst.top -o eq_npt_1fs.tpr -maxwarn 2
gmx mdrun -deffnm eq_npt_1fs -nb gpu
gmx grompp -f eq_npt_2fs.mdp -c eq_npt_1fs.gro -p syst.top -o eq_npt_2fs.tpr -maxwarn 2
gmx mdrun -deffnm eq_npt_2fs -nb gpu # 10 ns

# Production
gmx grompp -f prod.mdp -c eq_npt_2fs.gro -p syst.top -o prod.tpr
gmx mdrun -deffnm prod -nb gpu # 10 ns


Kinetic Energy

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/opc

#Using energy to extract the pressure and density of the system during the NPT prod MD
echo "Temperature" > options
echo "Kinetic-En." >> options
gmx energy -f eq_npt_2fs.edr -o npt_temp_kin.xvg < options # Berendsen
gmx energy -f prod.edr -o prod_temp_kin.xvg < options # Nose-Hoover

In [ ]:
%cd /content/opc

import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/opc")

ber = np.loadtxt(str('npt_temp_kin.xvg'), skiprows=25) # Berendsen
n_h = np.loadtxt(str('prod_temp_kin.xvg'), skiprows=25) # Nose-Hoover

print('Ber = ',"%.1f" % np.mean(ber[:,2]), '±', "%.1f" % np.std(ber[:,2]), 'K')
print('N-H = ',"%.1f" % np.mean(n_h[:,2]), '±', "%.1f" % np.std(n_h[:,2]), 'K')

print('Ber = ',"%.1f" % np.mean(ber[:,1]), '±', "%.1f" % np.std(ber[:,1]), 'kJ/mol')
print('N-H = ',"%.1f" % np.mean(n_h[:,1]), '±', "%.1f" % np.std(n_h[:,1]), 'kJ/mol')

pp.title('Comparison of Kinetic Energy')
pp.xlabel('Time (ps)')
pp.ylabel('Kinetic Energy (kJ/mol)')
pp.plot(ber[:,0], ber[:,1], linestyle='solid', linewidth='1', color='red')
pp.plot(n_h[:,0], n_h[:,1], linestyle='solid', linewidth='1', alpha=0.5, color='blue')
pp.show()

Density

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/opc


#Using energy to extract the pressure and density of the system during the NPT prod MD
echo "Pressure" > options
echo "Density" >> options
gmx energy -f prod.edr -o prod_press_dens.xvg < options

In [ ]:
%cd /content/opc

import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/opc")

ber = np.loadtxt(str('npt_press_dens.xvg'), skiprows=25) # Berendsen
p_r = np.loadtxt(str('prod_press_dens.xvg'), skiprows=25) # Parrinello-Rahman

print('Ber = ',"%.1f" % np.mean(ber[:,2]), '±', "%.1f" % np.std(ber[:,2]), 'kg/m^3')
print('P-R = ',"%.1f" % np.mean(p_r[:,2]), '±', "%.1f" % np.std(p_r[:,2]), 'kg/m^3')

Postprocessing

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC
cd /content/opc

echo "System" >> options
gmx trjconv -f prod.xtc -o prod.xtc -pbc mol -s prod.tpr < options

Creating an index file

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/opc

echo "ri 3" >> options
echo "4 & ! ri 3" >> options
echo "q" >> options
gmx make_ndx -f prod.gro -o ind.ndx < options

Radial distribution function

https://manual.gromacs.org/2024.0/reference-manual/analysis/radial-distribution-function.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/opc

# water around a water molecule
echo "5" > options # 1 water molecule
echo "6" >> options # water without "8"
gmx rdf -f prod.xtc -s prod.tpr -n ind.ndx -o rdf_wat1.xvg -pbc < options

# water around NA+
echo "2" > options # ion NA+
echo "4" >> options # water
gmx rdf -f prod.xtc -s prod.tpr -n ind.ndx -o rdf_na.xvg -pbc < options

# water around CL-
echo "3" > options # ion CL-
echo "4" >> options # water
gmx rdf -f prod.xtc -s prod.tpr -n ind.ndx -o rdf_cl.xvg -pbc < options

In [ ]:
%cd /content/opc

import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/opc")

na = np.loadtxt(str('rdf_na.xvg'), skiprows=25)*10
cl = np.loadtxt(str('rdf_cl.xvg'), skiprows=25)*10
w = np.loadtxt(str('rdf_wat1.xvg'), skiprows=25)*10

pp.title('RDF')
pp.xlabel('Distance, r (Å)')
pp.ylabel('g(r)')
pp.plot(na[:,0], na[:,1], linestyle='solid', linewidth='1', color='blue', label="around NA+")
pp.plot(cl[:,0], cl[:,1], linestyle='solid', linewidth='1', color='red', label="around CL-")
pp.plot(w[:,0], w[:,1], linestyle='solid', linewidth='1', color='black', label="around 1 H2O")

pp.legend(loc='upper right')
pp.show()

Mean squared displacement and Diffusion

https://manual.gromacs.org/2024.0/reference-manual/analysis/mean-square-displacement.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC
cd /content/opc

echo "4" > options # water
gmx msd -f prod.xtc -s prod.tpr -n ind.ndx -pbc -mol diff_w.xvg -o msd_w.xvg < options

echo "2" > options # ion NA+
gmx msd -f prod.xtc -s prod.tpr -n ind.ndx -pbc -mol diff_na.xvg -o msd_na.xvg < options

echo "3" > options # ion CL-
gmx msd -f prod.xtc -s prod.tpr -n ind.ndx -pbc -mol diff_cl.xvg -o msd_cl.xvg < options

In [ ]:
%cd /content/opc

import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/opc")

w = np.loadtxt(str('msd_w.xvg'), skiprows=25)
na = np.loadtxt(str('msd_na.xvg'), skiprows=25)
cl = np.loadtxt(str('msd_cl.xvg'), skiprows=25)

print("D (OPC water) = 2.2312 ± 0.0906 (e-5 cm^2/s)") # from MSD curve

pp.title('MSD')
pp.xlabel('Time lag (τ)')
pp.ylabel('MSD (nm$^2$/N)')
pp.plot(w[:,0], w[:,1], linestyle='solid', linewidth='1', color='black', label="water")
pp.plot(na[:,0], na[:,1], linestyle='solid', linewidth='1', color='blue', label="NA+")
pp.plot(cl[:,0], cl[:,1], linestyle='solid', linewidth='1', color='red', label="CL-")

pp.legend()
pp.show()

In [ ]:
%cd /content/opc

import matplotlib.pyplot as pp
import numpy as np
from pathlib import Path

#path = Path("/content/gdrive/MyDrive/MDSimulations/opc")

w = np.loadtxt(str('diff_w.xvg'), skiprows=25) # for each molecule

pp.title('MSD')
pp.xlabel('Molecule (#)')
pp.ylabel('Diffusion (e-5 cm^2/s)')
pp.plot(w[:,0], w[:,1], linestyle='solid', linewidth='1', color='red')
pp.show()

**That's all!**

# Homework

CHARMM-GUI

С помощью него генерируем топологию (CHARMM36m и AMBER) и систему для моделирования в GROMACS

0) Получаем дипептид в Pymol:

*   в командной строке Pymol пишем

      fab XY, pept

      alter all, chain="A"

      select all

      save XY.pdb, sele

где XY - два однобуквенных названия аминокислот

(https://www.chem.msu.su/rus/teaching/kolman/66.htm)

1) Register https://www.charmm-gui.org/ and Log in

2) переходим в раздел **Input Generator- Solution Builder**

3) **Protein Solution System** - Upload PDB File - Choose File - ваш **XY.pdb**, нажимаем Next Step:Select Model/Chain

4) Terminal group patching - оставляем NTER и CTER (т.е. NH3+ на N-конце и COO- на С-конце); нажимаем Next Step:Generate PDB

5) указываем форму и размер ячейки Waterbox Size Options: Rectangular, Specify Waterbox Size (X: 30 , Y: 30, Z: 30);

если **нет** заряженных аминокислот, нажимаем Calculate Solvent Composition и снимаем галочку с Include Ions,

если **есть** - удаляем KCl (нажать на "-") и добавляем NaCl (выбираем из списка и нажимаем Add Simple Ion Type), ставим концентрацию **0** (чтобы только нейтрализовать заряд аминокислот в системе) и нажимаем Calculate Solvent Composition;

нажимаем Next Step:Solvate Molecule

6) Next Step:Setup Periodic Boundary Condition

7) Выбираем FF **CHARMM36m**, Input Generation Options: **GROMACS**, нажимаем Next Step:Generate Equilibration and Dynamics inputs

8) Нажимаем download.tgz

9) Создаём папки MD_chm36m, MD_amber в Мой диск (Google Drive)

10) Извлекаем/распаковываем из скачанного charmm-gui.tgz папку gromacs. Из gromacs загружаем на Мой диск в папку MD_chm36m: папку toppar и файлы step3_input.gro, topol.top

9) заходим в User Profile (в правом верхнем углу около Logout) - Job IDs, нажимаем на номер Job ID, нажимаем **Go**, где solution 3

10) Выбираем FF - **AMBER**, Water - **OPC**, Input Generation Options: **GROMACS**, нажимаем Next Step:Generate Equilibration and Dynamics inputs

11) Нажимаем download.tgz

12) Извлекаем/распаковываем из скачанного charmm-gui.tgz папку gromacs. Из gromacs загружаем на Мой диск в папку MD_amber: папку toppar и файлы step3_input.gro, topol.top

13) В Colab выбираем Runtime type - T4 GPU, нажимаем Connect to a hosted runtime T4

14) Выполняем следующие ниже ячейки



In [ ]:
#Downloading GROMACS 2024

!wget https://ftp.gromacs.org/gromacs/gromacs-2024.tar.gz

In [ ]:
%%bash

#Extracting and Installing GROMACS 2024 onto a user-defined folder
#First, we extract the downloaded folder containing GROMACS-2024

tar xfz gromacs-2024.tar.gz # tar - archive, gz - gzip compression
echo "GROMACS extraction completed"

#Then, we create a "build" folder inside the extracted folder

cd gromacs-2024

mkdir build

#Next, we setup our GROMACS 2024 installation options, including GPU and a user-defined folder

cd build

cmake .. -DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=CUDA -DCMAKE_INSTALL_PREFIX=/content/gromacs-24
echo "GROMACS set up completed"

In [ ]:
%%bash

cd /content/gromacs-2024/build

# ~ 1 час компилируется

make
#echo "GROMACS building completed"

make install
#echo "GROMACS installation completed. Please check if any errors occurred during installation"

In [ ]:
%%bash

##Checking that GROMACS was successfully installed

source /content/gromacs-24/bin/GMXRC

gmx -h

Монтируем Google drive к текущей сессии Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Загружаем папки из Google Drive на Colab

In [ ]:
import shutil
from pathlib import Path
import os

MD_chm36m = Path("/content/gdrive/My Drive/MD_chm36m")
MD_amber = Path("/content/gdrive/My Drive/MD_amber")

shutil.copytree(str(MD_chm36m), str('/content/MD_chm36m'))
shutil.copytree(str(MD_amber), str('/content/MD_amber'))

**FF Charmm (CHARMM36m)**

https://doi.org/10.1038/nmeth.4067

Минимизация, уравновешивание

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_chm36m

wget https://zenodo.org/records/10822792/files/min.mdp
wget https://zenodo.org/records/10822792/files/eq_nvt_1fs.mdp
wget https://zenodo.org/records/10822792/files/eq_npt_1fs.mdp
wget https://zenodo.org/records/10822792/files/prod.mdp

# Minimization
gmx grompp -f min.mdp -c step3_input.gro -p topol.top -o min.tpr
gmx mdrun -deffnm min -nb gpu

#Thermalization and Equilibration
# NVT
gmx grompp -f eq_nvt_1fs.mdp -c min.gro -p topol.top -o eq_nvt_1fs.tpr -maxwarn 1
gmx mdrun -deffnm eq_nvt_1fs -nb gpu
# NPT
gmx grompp -f eq_npt_1fs.mdp -c eq_nvt_1fs.gro -p topol.top -o eq_npt_1fs.tpr -maxwarn 2
gmx mdrun -deffnm eq_npt_1fs -nb gpu

MD production (10 ns) ~ 25 минут

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_chm36m

# Production
gmx grompp -f prod.mdp -c eq_npt_1fs.gro -p topol.top -o prod.tpr
gmx mdrun -deffnm prod -nb gpu # 10 ns

Восстанавливаем целостность молекул на границе ячейки для дальнейших анализов

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_chm36m

echo "System" >> options
gmx trjconv -f prod.xtc -o prod.xtc -pbc mol -s prod.tpr < options

Создаём файл с индексами атомов, где они разбиты на группы

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_chm36m

echo "q" > options
gmx make_ndx -f prod.gro -o ind.ndx < options

Анализируем RMSD определённой группы атомов относительно стартовой структуры (из Pymol)

https://manual.gromacs.org/current/reference-manual/analysis/rmsd.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_chm36m

echo "Protein-H" > options
echo "Protein-H" >> options
gmx rms -s min.tpr -f prod.xtc -n ind.ndx -o rmsd.xvg -fit rot+trans < options # rmsd дипептида (без водородов, т.е. только тяжёлые атомы) относительно стартовой структуры (из pymol)

In [ ]:
%cd /content/MD_chm36m

import numpy as np
import matplotlib.pyplot as pp

data = np.loadtxt('rmsd.xvg', skiprows=25)

print('RMSD =',"%.1f" % np.mean(data[:,1]*10), '±', "%.1f" % np.std(data[:,1]*10), 'Å')

pp.title('RMSD (Protein-H) = {} ± {} Å'.format(np.round(np.mean(data[:,1]*10),1), np.round(np.std(data[:,1]*10),1)))
pp.xlabel('Time (ps)')
pp.ylabel('RMSD (Å)')
pp.plot(data[:,0], data[:,1]*10, linestyle='solid', linewidth='1', color='blue')
pp.savefig('rmsd.png', dpi=300)
pp.show()

Радиус гирации

https://manual.gromacs.org/documentation/2019-rc1/reference-manual/analysis/radius-of-gyration.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_chm36m

echo "2" > options
gmx gyrate -f prod.xtc -s prod.tpr -n ind.ndx -o gyrate.xvg < options

In [ ]:
%cd /content/MD_chm36m

import numpy as np
import matplotlib.pyplot as pp

data = np.loadtxt('gyrate.xvg', skiprows=27)

print('Radius =',"%.1f" % np.mean(data[:,1]*10), '±', "%.1f" % np.std(data[:,1]*10), 'Å')

pp.title('Radius of gyration (Protein-H) = {} ± {} Å'.format(np.round(np.mean(data[:,1]*10),1), np.round(np.std(data[:,1]*10),1)))
pp.xlabel('Time (ps)')
pp.ylabel(r'R$_g$ (Å)')
pp.plot(data[:,0], data[:,1]*10, linestyle='solid', linewidth='1', color='blue')
pp.savefig('radius.png', dpi=300)
pp.show()

Диффузия

https://manual.gromacs.org/current/reference-manual/analysis/mean-square-displacement.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_chm36m

echo "2" > options
gmx msd -f prod.xtc -s prod.tpr -n ind.ndx -pbc -o msd.xvg < options

In [ ]:
%cd /content/MD_chm36m

import numpy as np
import matplotlib.pyplot as pp

data = np.loadtxt('msd.xvg', skiprows=25)

A = np.vstack([data[200:600,0], np.ones(len(data[200:600,0]))]).T
y = data[200:600,1][:, np.newaxis]
alpha = np.dot((np.dot(np.linalg.inv(np.dot(A.T,A)),A.T)),y)

print('D = ',str(np.round(alpha[0][0]*1000/6,1)),'e-5 cm^2/s')

pp.title('MSD (Protein-H) and Fit (2-8 ns), D = {} e-5 cm^2/s'.format(np.round(alpha[0][0]*1000/6,1)))
pp.xlabel('Time lag (ps)')
pp.ylabel(r'MSD (nm$^2$)')
pp.plot(data[:,0], data[:,1], linestyle='solid', linewidth='1', color='blue', label="MSD")
pp.plot(data[:,0], alpha[0]*data[:,0] + alpha[1], linestyle='solid', linewidth='1', color='red', label="Least Squares Fit")
pp.legend()
pp.savefig('msd.png', dpi=300)
pp.show()

In [ ]:
import shutil
from pathlib import Path
import os

MD_chm36m = Path("/content/gdrive/My Drive/MD_chm36m/")

shutil.copy(str('/content/MD_chm36m/min.gro'), str(MD_chm36m))
shutil.copy(str('/content/MD_chm36m/prod.tpr'), str(MD_chm36m))
shutil.copy(str('/content/MD_chm36m/prod.xtc'), str(MD_chm36m))
shutil.copy(str('/content/MD_chm36m/ind.ndx'), str(MD_chm36m))

shutil.copy(str('/content/MD_chm36m/rmsd.png'), str(MD_chm36m))
shutil.copy(str('/content/MD_chm36m/radius.png'), str(MD_chm36m))
shutil.copy(str('/content/MD_chm36m/msd.png'), str(MD_chm36m))

# файлы могут не сразу появиться на Диске

**FF Amber (ff19SB)**

https://chemrxiv.org/engage/api-gateway/chemrxiv/assets/orp/resource/item/60c7426b0f50db0f23395cd7/original/ff19sb-amino-acid-specific-protein-backbone-parameters-trained-against-quantum-mechanics-energy-surfaces-in-solution.pdf

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_amber

wget https://zenodo.org/records/10822792/files/min.mdp
wget https://zenodo.org/records/10822792/files/eq_nvt_1fs.mdp
wget https://zenodo.org/records/10822792/files/eq_npt_1fs.mdp
wget https://zenodo.org/records/10822792/files/prod.mdp

# Minimization
gmx grompp -f min.mdp -c step3_input.gro -p topol.top -o min.tpr
gmx mdrun -deffnm min -nb gpu

#Thermalization and Equilibration
# NVT
gmx grompp -f eq_nvt_1fs.mdp -c min.gro -p topol.top -o eq_nvt_1fs.tpr -maxwarn 1
gmx mdrun -deffnm eq_nvt_1fs -nb gpu
# NPT
gmx grompp -f eq_npt_1fs.mdp -c eq_nvt_1fs.gro -p topol.top -o eq_npt_1fs.tpr -maxwarn 2
gmx mdrun -deffnm eq_npt_1fs -nb gpu

MD production (10 ns) ~ 40 минут

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_amber

# Production 10 ns
gmx grompp -f prod.mdp -c eq_npt_1fs.gro -p topol.top -o prod.tpr
gmx mdrun -deffnm prod -nb gpu

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_amber

echo "System" >> options
gmx trjconv -f prod.xtc -o prod.xtc -pbc mol -s prod.tpr < options

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_amber

echo "q" > options
gmx make_ndx -f prod.gro -o ind.ndx < options

Анализируем RMSD определённой группы атомов относительно стартовой структуры (из Pymol)

https://manual.gromacs.org/current/reference-manual/analysis/rmsd.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_amber

echo "Protein-H" > options
echo "Protein-H" >> options
gmx rms -s min.tpr -f prod.xtc -n ind.ndx -o rmsd.xvg -fit rot+trans < options # rmsd дипептида (без водородов, т.е. только тяжёлые атомы) относительно стартовой структуры (из pymol)

In [ ]:
%cd /content/MD_amber

import numpy as np
import matplotlib.pyplot as pp

data = np.loadtxt('rmsd.xvg', skiprows=25)

print('RMSD =',"%.1f" % np.mean(data[:,1]*10), '±', "%.1f" % np.std(data[:,1]*10), 'Å')

pp.title('RMSD (Protein-H) = {} ± {} Å'.format(np.round(np.mean(data[:,1]*10),1), np.round(np.std(data[:,1]*10),1)))
pp.xlabel('Time (ps)')
pp.ylabel('RMSD (Å)')
pp.plot(data[:,0], data[:,1]*10, linestyle='solid', linewidth='1', color='blue')
pp.savefig('rmsd.png', dpi=300)
pp.show()

Радиус гирации

https://manual.gromacs.org/documentation/2019-rc1/reference-manual/analysis/radius-of-gyration.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_amber

echo "2" > options
gmx gyrate -f prod.xtc -s prod.tpr -n ind.ndx -o gyrate.xvg < options

In [ ]:
%cd /content/MD_amber

import numpy as np
import matplotlib.pyplot as pp

data = np.loadtxt('gyrate.xvg', skiprows=27)

print('Radius =',"%.1f" % np.mean(data[:,1]*10), '±', "%.1f" % np.std(data[:,1]*10), 'Å')

pp.title('Radius of gyration (Protein-H) = {} ± {} Å'.format(np.round(np.mean(data[:,1]*10),1), np.round(np.std(data[:,1]*10),1)))
pp.xlabel('Time (ps)')
pp.ylabel(r'R$_g$ (Å)')
pp.plot(data[:,0], data[:,1]*10, linestyle='solid', linewidth='1', color='blue')
pp.savefig('radius.png', dpi=300)
pp.show()

Диффузия

https://manual.gromacs.org/current/reference-manual/analysis/mean-square-displacement.html

In [ ]:
%%bash

source /content/gromacs-24/bin/GMXRC

cd /content/MD_amber

echo "2" > options
gmx msd -f prod.xtc -s prod.tpr -n ind.ndx -pbc -o msd.xvg < options

In [ ]:
%cd /content/MD_amber

import numpy as np
import matplotlib.pyplot as pp

data = np.loadtxt('msd.xvg', skiprows=25)

A = np.vstack([data[200:600,0], np.ones(len(data[200:600,0]))]).T
y = data[200:600,1][:, np.newaxis]
alpha = np.dot((np.dot(np.linalg.inv(np.dot(A.T,A)),A.T)),y)

print('D = ',str(np.round(alpha[0][0]*1000/6,1)),'e-5 cm^2/s')

pp.title('MSD (Protein-H) and Fit (2-8 ns), D = {} e-5 cm^2/s'.format(np.round(alpha[0][0]*1000/6,1)))
pp.xlabel('Time lag (ps)')
pp.ylabel(r'MSD (nm$^2$)')
pp.plot(data[:,0], data[:,1], linestyle='solid', linewidth='1', color='blue', label="MSD")
pp.plot(data[:,0], alpha[0]*data[:,0] + alpha[1], linestyle='solid', linewidth='1', color='red', label="Least Squares Fit")
pp.legend()
pp.savefig('msd.png', dpi=300)
pp.show()

In [ ]:
import shutil
from pathlib import Path
import os

MD_amber = Path("/content/gdrive/My Drive/MD_amber")

shutil.copy(str('/content/MD_amber/min.gro'), str(MD_amber))
shutil.copy(str('/content/MD_amber/prod.tpr'), str(MD_amber))
shutil.copy(str('/content/MD_amber/prod.xtc'), str(MD_amber))
shutil.copy(str('/content/MD_chm36m/ind.ndx'), str(MD_amber))

shutil.copy(str('/content/MD_amber/rmsd.png'), str(MD_amber))
shutil.copy(str('/content/MD_amber/radius.png'), str(MD_amber))
shutil.copy(str('/content/MD_amber/msd.png'), str(MD_amber))

Перед отключением проверьте, что файлы появились на Google диске.